# Example: Neural Networks – Relaxing walk

Before runnning this notebook, make sure that all neccesary libraries are installed:

In [13]:
using Flux
using Random
using Gogeta
using Gurobi
using JuMP
using Plots
using Revise
using QuasiMonteCarlo

In this example, we are going to introduce the function `optimize_by_walking!()` that can be used to optimize MILP formulation of the neural network faster than optimizing the formulation directly. To understand the method in detail, see Tong, J et al. (2024)

## Innitialize neural network with random weights

In [14]:
dimension = 2

begin
    Random.seed!(12345);

    NN_model = Chain(
        Dense(dimension => 100, relu),
        Dense(100 => 100, relu),
        Dense(100 => 1)
    )
end

Chain(
  Dense(2 => 100, relu),                # 300 parameters
  Dense(100 => 100, relu),              # 10_100 parameters
  Dense(100 => 1),                      # 101 parameters
)                   # Total: 6 arrays, 10_501 parameters, 41.395 KiB.

## Formulate  NN as a MILP 

We set up upper and lower bounds for the variables in which our MILP formulation is guranteed to output the same values as the original NN

In [15]:
init_U = [5.0, 5.0];
init_L = [-5.0, -5.0];

Formulate MILP model with fast bound tightening. Set objective funciton of the model as a maximiztion of the output neuron.

In [16]:
# Formulate the MIP with heuristic bound tightening
jump_model = Model(Gurobi.Optimizer)
set_silent(jump_model)
NN_formulate!(jump_model, NN_model, init_U, init_L; bound_tightening="fast");

last_layer, _ = maximum(keys(jump_model[:x].data))
@objective(jump_model, Max, jump_model[:x][last_layer, 1])

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20


x[3,1]

`set_solver!()` should be specified

In [17]:
function set_solver!(jump)
    set_optimizer(jump, Gurobi.Optimizer)
    set_silent(jump)
end

set_solver! (generic function with 1 method)

## Optimize by relax walking

In order to use optimization using relax walking, you just need to call function `optimize_by_walking!()` with the following input parameters:
- jump_model – empty jump model
- NN_model – the neural net that you want to represent as a jump model
- init_U, init_L – upper and lower bounds in which our solution is guranteed to generate the same result as NN

In [18]:
x_opt, opt = optimize_by_walking!(jump_model, NN_model, init_U, init_L)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20


LoadError: MethodError: no method matching local_search(::Vector{Float64}, ::Model, ::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}, ::Vector{Float64}, ::Vector{Float64})

[0mClosest candidates are:
[0m  local_search(::Any, ::Any, ::Any, ::Any; max_iter, epsilon, show_path, logging, tolerance)
[0m[90m   @[39m [35mGogeta[39m [90m~/.julia/packages/Gogeta/iJMCg/src/neural_networks/[39m[90m[4mrelaxing_walk.jl:171[24m[39m


It returns optimal solution along with the optimal objective function.